1. Buatlah data text representasi dari dataset tweet_sentiment pilkada DKI 2017 menggunakan metode Bag Of Words dan TF IDF dengan terlebih dahulu melakukan pembersihan data teks.
2. Simpan dalam file csv dengan nama Pilkada_DKI_BoW.csv dan Pilkada_DKI_TFIDF.csv.

**Import Library**

In [128]:
import numpy as np
import pandas as pd
import re
import nltk
!pip install nlp-id
from nlp_id.lemmatizer import Lemmatizer
!pip install PySastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nlp_id.stopword import StopWord
from nltk import sent_tokenize
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import math
from sklearn.preprocessing import normalize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


****

**Import Dataset**

In [62]:
#Load Dataset
df=pd.read_csv('https://github.com/TyasArgani/PSDS-Kelas-Mahir/raw/main/dataset_tweet_sentiment_pilkada_DKI_2017.csv')
df.head()

,Id,Sentiment,Pasangan Calon,Text Tweet
0,1,negative,Agus-Sylvi,Banyak akun kloning seolah2 pendukung #agussil...
1,2,negative,Agus-Sylvi,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,3,negative,Agus-Sylvi,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,4,negative,Agus-Sylvi,Kasian oh kasian dengan peluru 1milyar untuk t...
4,5,negative,Agus-Sylvi,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...


**Menyimpan Hasil Review dalam DataFrame**

In [63]:
data=pd.DataFrame(df['Text Tweet'])
data

,Text Tweet
0,Banyak akun kloning seolah2 pendukung #agussil...
1,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,Kasian oh kasian dengan peluru 1milyar untuk t...
4,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...
...,...
895,"Kali saja bpk @aniesbaswedan @sandiuno lihat, ..."
896,Kita harus dapat merangkul semua orang tanpa b...
897,Ini jagoanku dibidang digital <Smiling Face Wi...
898,#PesanBijak #OkeOce #GubernurGu3 ...


**Mengambil Series Data Teks Review**

In [64]:
teks=data['Text Tweet']
teks

0      Banyak akun kloning seolah2 pendukung #agussil...
1      #agussilvy bicara apa kasihan yaa...lap itu ai...
2      Kalau aku sih gak nunggu hasil akhir QC tp lag...
3      Kasian oh kasian dengan peluru 1milyar untuk t...
4      Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...
                             ...                        
895    Kali saja bpk @aniesbaswedan @sandiuno lihat, ...
896    Kita harus dapat merangkul semua orang tanpa b...
897    Ini jagoanku dibidang digital <Smiling Face Wi...
898                 #PesanBijak #OkeOce #GubernurGu3 ...
899    Sandiaga: Bangun Rumah DP 0% Lebih Simpel Diba...
Name: Text Tweet, Length: 900, dtype: object

**Membuat Fungsi untuk Melakukan Case Folding**

In [65]:
import re, string, unicodedata 
def Case_Folding(text):
    # Hapus non-ascii
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # Menghapus Tanda Baca
    text = re.sub(r'[^\w]|_',' ', text)
    
    # Menghapus Angka
    text = re.sub("\S*\d\S*", "", text).strip()
    text = re.sub(r"\b\d+\b", " ", text)
    
    # Mengubah text menjadi lowercase
    text = text.lower()
    
    # Menghapus white space
    text = re.sub('[\s]+', ' ', text)
    
    return text

**Membuat Fungsi untuk Melakukan Lemmatization**

In [66]:
def lemma(text):
  lemmatizer = Lemmatizer()
  text=lemmatizer.lemmatize(text)
  return text

**Membuat Fungsi untuk Melakukan Stemming**

In [67]:
def stemming(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  text=stemmer.stem(text)
  return text

**Membuat Fungsi untuk Melakukan Slang Words**

In [68]:
slang_dictionary = pd.read_csv('https://raw.githubusercontent.com/TyasArgani/PSDS-Kelas-Mahir/main/colloquial-indonesian-lexicon.csv')
slang_dict = pd.Series(slang_dictionary['formal'].values, index=slang_dictionary['slang']).to_dict()

def Slangwords(text):
  for word in text.split():
    if word in slang_dict.keys():
      text = text.replace(word, slang_dict[word])
  return text

**Membuat Fungsi untuk Melakukan Stopword**

In [71]:
def stopword(text):
  stopword = StopWord()
  text=stopword.remove_stopword(text)
  return text

**Menerapkan Semua Langkah**

In [70]:
data['Text Tweet Processed']=''

for i, row in data.iterrows():
  content=teks[i]
  result=Case_Folding(content)
  result=lemma(result)
  result=stemming(result)
  result=Slangwords(result)
  result=stopword(result)
  data['Text Tweet Processed'][i]=result

data[['Text Tweet','Text Tweet Processed']]

,Text Tweet,Text Tweet Processed
0,Banyak akun kloning seolah2 pendukung #agussil...,akun kloning dukung agussilvy serang paslon an...
1,#agussilvy bicara apa kasihan yaa...lap itu ai...,agussilvy bicara kasihan lap air mata wkwkwkwk
2,Kalau aku sih gak nunggu hasil akhir QC tp lag...,enggak memenunggu hasil qc memenunggu motif cu...
3,Kasian oh kasian dengan peluru 1milyar untuk t...,kasihh oh kasihh peluru rw agussilvy mempan me...
4,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...,maaf dukung agussilvy hayo dukung aniessandi p...
...,...,...
895,"Kali saja bpk @aniesbaswedan @sandiuno lihat, ...",bpk aniesbaswedan sandiuno lihat rspun selfie ...
896,Kita harus dapat merangkul semua orang tanpa b...,rangkul orang batas usia kelamin okeoce ok han...
897,Ini jagoanku dibidang digital <Smiling Face Wi...,jago bidang digital smiling face with sunglass...
898,#PesanBijak #OkeOce #GubernurGu3 ...,pesanbijak okeoce


# **Bag Of Word**

**Menyimpan Hasil Review dalam DataFrame**

In [76]:
df=pd.DataFrame(data['Text Tweet Processed'])
df

,Text Tweet Processed
0,akun kloning dukung agussilvy serang paslon an...
1,agussilvy bicara kasihan lap air mata wkwkwkwk
2,enggak memenunggu hasil qc memenunggu motif cu...
3,kasihh oh kasihh peluru rw agussilvy mempan me...
4,maaf dukung agussilvy hayo dukung aniessandi p...
...,...
895,bpk aniesbaswedan sandiuno lihat rspun selfie ...
896,rangkul orang batas usia kelamin okeoce ok han...
897,jago bidang digital smiling face with sunglass...
898,pesanbijak okeoce


**Mengambil Series Data Teks Review**

In [86]:
df_teks=data['Text Tweet Processed']
df_teks

0      akun kloning dukung agussilvy serang paslon an...
1         agussilvy bicara kasihan lap air mata wkwkwkwk
2      enggak memenunggu hasil qc memenunggu motif cu...
3      kasihh oh kasihh peluru rw agussilvy mempan me...
4      maaf dukung agussilvy hayo dukung aniessandi p...
                             ...                        
895    bpk aniesbaswedan sandiuno lihat rspun selfie ...
896    rangkul orang batas usia kelamin okeoce ok han...
897    jago bidang digital smiling face with sunglass...
898                                    pesanbijak okeoce
899    sandiaga bangun rumah dp simpel banding tol ci...
Name: Text Tweet Processed, Length: 900, dtype: object

**Membuat Fungsi `Bag_Of_Words()`**

Membuat fungsi BOW dengan menerapkan semua langkah.

In [87]:
def Bag_Of_Words(df):
    # Membuat list kosa kata dari seluruh teks input
    kosa_kata = set()
    for i in df_teks:
        token_kal=set(word_tokenize(i.lower()))
        kosa_kata=set.union(kosa_kata,token_kal)
    kosa_kata = sorted(kosa_kata)
    # Membuat dictionary kata-kata
    indeks_kata = {}
    for kata in kosa_kata:
        indeks_kata[kata] = 0
    # Jumlah kosa kata(untuk kolom)
    jumlah_kosa_kata = len(kosa_kata)
    # Membagun matriks kata
    vektor_kata=np.array([])
    for baris_teks in df_teks:
        baris_kata = indeks_kata.copy()
        for kata in baris_teks.lower().split():
            baris_kata[kata]=+1
        baris_kata = np.array(list(baris_kata.values()))  
        vektor_kata = np.concatenate((vektor_kata,baris_kata), axis=0)
    matriks_kata = vektor_kata.reshape(df.shape[0], jumlah_kosa_kata)
    #Membuat DataFrame Bag Of Words    
    df_bow = pd.DataFrame(matriks_kata, columns=kosa_kata, index=df_teks.tolist())
    df_bow = df_bow.astype('int')
    return df_bow

**Implementasi Fungsi `Bag_Of_Words()`**

In [93]:
df_BoW=Bag_Of_Words(df)
df_BoW

,a,aa,aagym,abaaah,abal,abang,abdi,absurd,abunawas,aburizal,abused,acara,acu,adaaqua,adalahkita,addiems,adekku,adil,adjierimbawan,adjrot,agam,agama,agllusylludhoyono,agus,agusharimurtiyudhoyono,agussilvy,agussylvi,agusyudhoyono,ah,aher,ahli,ahmadfuadanwar,ahok,ahokbtp,ahokdicintairakyat,ahokdjarot,ahokdjarota,ahokdjarotdirosi,ahokdjarotlebihbaik,ahokdjarotmenang,...,wasekjen,waspada,wawancara,wearethenextleader,weary,weedbegoodtogether,welcome,wiiih,wilayah,wingstourinjakarta,wirausaha,wisata,wisdom,with,without,wkkwwkkwk,wkwkwk,wkwkwkwk,wkwkwkwkkwkk,wpap,wujud,yahudi,yangbikinberantem,ye,yes,yme,ynwa,yogia,youtu,youtube,yra,yudhoyono,yuk,yusuf,zalim,zaraz,zarazettirazr,zarazettirazz,zipper,zona
akun kloning dukung agussilvy serang paslon aniessandi opini argumen pmbenaran kecoh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
agussilvy bicara kasihan lap air mata wkwkwkwk,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
enggak memenunggu hasil qc memenunggu motif cuit sbyudhoyono kayak pasca agussilvy nyungsep,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
kasihh oh kasihh peluru rw agussilvy mempan menangin pilkada quickcount,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
maaf dukung agussilvy hayo dukung aniessandi putar ronavioleta netizentofa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bpk aniesbaswedan sandiuno lihat rspun selfie okeoce ok hand debatpilkadadki ahokpanikahokkalah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
rangkul orang batas usia kelamin okeoce ok hand salambersama pks temanahok victory hand,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
jago bidang digital smiling face with sunglasses ok hand thonyleong com okeoce salambersama,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
pesanbijak okeoce,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Menyimpan DataFrame ke dalam CSV**

In [140]:
df_BoW.to_csv('Pilkada_DKI_BoW.csv')

# **TF-IDF**

**Menyimpan Hasil Review dalam DataFrame**

In [90]:
df=pd.DataFrame(data['Text Tweet Processed'])
df

,Text Tweet Processed
0,akun kloning dukung agussilvy serang paslon an...
1,agussilvy bicara kasihan lap air mata wkwkwkwk
2,enggak memenunggu hasil qc memenunggu motif cu...
3,kasihh oh kasihh peluru rw agussilvy mempan me...
4,maaf dukung agussilvy hayo dukung aniessandi p...
...,...
895,bpk aniesbaswedan sandiuno lihat rspun selfie ...
896,rangkul orang batas usia kelamin okeoce ok han...
897,jago bidang digital smiling face with sunglass...
898,pesanbijak okeoce


**Mengambil Series Data Teks Review**

In [ ]:
df_teks=data['Text Tweet Processed']
df_teks

**Membuat Fungsi `TFIDF()`**

In [137]:
def TF_IDF(df):
    # Membuat list kosa kata dari seluruh teks input
    kosa_kata = set()
    for i in df_teks:
        token_kal=set(word_tokenize(i.lower()))
        kosa_kata=set.union(kosa_kata,token_kal)
    kosa_kata = sorted(kosa_kata)
    # Membuat dictionary kata-kata
    indeks_kata = {}
    for kata in kosa_kata:
        indeks_kata[kata] = 0
    # Jumlah kosa kata(untuk kolom)
    jumlah_kosa_kata = len(kosa_kata)
    # Membagun matriks kata TF
    vektor_TF=np.array([])
    for baris_teks in df_teks:
        baris_kata = indeks_kata.copy()
        split_baris = baris_teks.lower().split()
        for kata in baris_teks.lower().split():
            baris_kata[kata]=+1
        baris_kata = np.array(list(baris_kata.values()))
        baris_kata = baris_kata/len(split_baris)
        vektor_TF = np.concatenate((vektor_TF,baris_kata), axis=0)
    matriks_TF = vektor_TF.reshape(df.shape[0], jumlah_kosa_kata)
    # Membagun dataframe kata TF
    df_TF = pd.DataFrame(matriks_TF, columns=kosa_kata, index=df_teks.tolist())
    # Menghitung baris IDF
    baris_IDF=[]
    for kolom in df_TF.columns :
        N = len(df_teks)
        kal_kata = len(df_TF[df_TF[kolom]>0])
        baris_idf = (math.log((1+N)/(1+kal_kata)))+1  
        baris_IDF.append(baris_idf)
    # Membuat matriks TF-IDF
    vektor_TFIDF=np.array([])
    for baris in matriks_TF:
        baris_tfidf=np.multiply(baris,baris_IDF)
        vektor_TFIDF=np.concatenate((vektor_TFIDF,baris_tfidf), axis=0)
    matriks_TFIDF = vektor_TFIDF.reshape(df.shape[0], jumlah_kosa_kata)
    matriks_TFIDF = normalize(matriks_TFIDF)
    # Membuat dataframe TF-IDF
    df_TF_IDF = pd.DataFrame(matriks_TFIDF, columns=kosa_kata, index=df_teks.tolist())
    return df_TF_IDF

**Implementasi Fungsi `TF_IDF()`**

In [138]:
df_TF_IDF=TF_IDF(df)
df_TF_IDF

,a,aa,aagym,abaaah,abal,abang,abdi,absurd,abunawas,aburizal,abused,acara,acu,adaaqua,adalahkita,addiems,adekku,adil,adjierimbawan,adjrot,agam,agama,agllusylludhoyono,agus,agusharimurtiyudhoyono,agussilvy,agussylvi,agusyudhoyono,ah,aher,ahli,ahmadfuadanwar,ahok,ahokbtp,ahokdicintairakyat,ahokdjarot,ahokdjarota,ahokdjarotdirosi,ahokdjarotlebihbaik,ahokdjarotmenang,...,wasekjen,waspada,wawancara,wearethenextleader,weary,weedbegoodtogether,welcome,wiiih,wilayah,wingstourinjakarta,wirausaha,wisata,wisdom,with,without,wkkwwkkwk,wkwkwk,wkwkwkwk,wkwkwkwkkwkk,wpap,wujud,yahudi,yangbikinberantem,ye,yes,yme,ynwa,yogia,youtu,youtube,yra,yudhoyono,yuk,yusuf,zalim,zaraz,zarazettirazr,zarazettirazz,zipper,zona
akun kloning dukung agussilvy serang paslon aniessandi opini argumen pmbenaran kecoh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.246002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
agussilvy bicara kasihan lap air mata wkwkwkwk,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.298609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.43696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
enggak memenunggu hasil qc memenunggu motif cuit sbyudhoyono kayak pasca agussilvy nyungsep,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.246506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
kasihh oh kasihh peluru rw agussilvy mempan menangin pilkada quickcount,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.258239,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
maaf dukung agussilvy hayo dukung aniessandi putar ronavioleta netizentofa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.308951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bpk aniesbaswedan sandiuno lihat rspun selfie okeoce ok hand debatpilkadadki ahokpanikahokkalah,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rangkul orang batas usia kelamin okeoce ok hand salambersama pks temanahok victory hand,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
jago bidang digital smiling face with sunglass

**Menyimpan DataFrame ke dalam CSV**

In [141]:
df_TF_IDF.to_csv('Pilkada_DKI_TFIDF.csv')